In [1]:
### NP - METHODS TO GET PAGE CONTENT USING SELENIUM AND RETURN AS SOUP

# Needs a proper method to sleep during execution

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service # del netinkamos driverio versijos galima autoamtiskai sutvarkjyt
#from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
from typing import Optional

from selenium.webdriver.common.by import By


country: str = "LT"
query: str = f"https://www.nordpoolgroup.com/en/Market-data1/Dayahead/Area-Prices/{country}/Hourly/?view=table"


def _get_page_content(query: str) -> Optional[BeautifulSoup]:
        """Method to get needed content from search result page or whatever related page."""
        
        chrome_options = webdriver.ChromeOptions()
        # this will disable image loading
        chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
        
        driver = webdriver.Chrome(chrome_options=chrome_options)   #options=options,    
        driver.get(query)
        time.sleep(3)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        checker = soup.find("div", id="error-content")
                
        
        if checker is None:            
            return soup
        elif checker.h1.text == "Important information":
            print("Wrong input.")
            
        driver.quit()
    



## FIXai
# padaryti, kad neissoktu narsykle
# padaryti papildoma patikra puslapio statusui (200, 404 etc)




In [4]:
### NP MYSQL METHODS

import os
import datetime
import mysql.connector
from typing import List


def _push_to_database(mysql_query: str, params: tuple = None) -> None:
    """Methods retrieves year id from database. If nno such year in database returns NONE."""
    try:
        conn = mysql.connector.connect(
        user=os.environ.get("DB_USER"),
        password=os.environ.get("DB_PASSWORD"),
        database=os.environ.get("DATABASE"),
        port=int(os.environ.get("DB_PORT")),
        host=os.environ.get("DB_HOST"),
        auth_plugin=os.environ.get("AUTH_PLUGIN")
        ) 

        cur = conn.cursor(prepared=True)        
        cur.execute(mysql_query, params)
        conn.commit()
        print("Records pushed succesfully") # VIETA LOGINIMUI
        

    except mysql.connector.Error as error:
        print("Failed to execute MySQL query {}".format(error)) # VIETA LOGINIMUI

    finally:
        if conn.is_connected():
            cur.close()
            conn.close()
            print("MySQL conn is closed.") #VIETA LOGINIMUI


def _check_date_in_database(all_dates) -> List:
    """Method checks if date exists in database. Records wich does not exists returns as list."""
    filtered_dates: List = []

    try:
        conn = mysql.connector.connect(
        user=os.environ.get("DB_USER"),
        password=os.environ.get("DB_PASSWORD"),
        database=os.environ.get("DATABASE"),
        port=int(os.environ.get("DB_PORT")),
        host=os.environ.get("DB_HOST"),
        auth_plugin=os.environ.get("AUTH_PLUGIN")
        ) 

        cur = conn.cursor()        
        for date in all_dates[1:]:
            # Pakonvertuoja is 08-12-2022 i 2022-12-08 + pavercia i date tipa
            formated_date = datetime.strptime(date.text, '%d-%m-%Y').date()
            mysql_query: str = """SELECT date_of_price FROM nordpool.HourlyPricesLT where date_of_price=%s;"""
            cur.execute(mysql_query, (formated_date,))
            cur.fetchone()
            row_count = cur.rowcount
            if row_count < 0:   
                filtered_dates.append(formated_date)
        filtered_dates.reverse()
        return filtered_dates
        
    except mysql.connector.Error as error:
        print("Failed to execute MySQL query {}".format(error)) # VIETA LOGINIMUI

    finally:
        if conn.is_connected():
            cur.close()
            conn.close()
            print("MySQL conn for checks is closed.") #VIETA LOGINIMUI




def _check_date_in_database___old(params: str) -> bool:
    """Method checks if date exists in database. Returns boolean."""

    try:
        conn = mysql.connector.connect(
        user=os.environ.get("DB_USER"),
        password=os.environ.get("DB_PASSWORD"),
        database=os.environ.get("DATABASE"),
        port=int(os.environ.get("DB_PORT")),
        host=os.environ.get("DB_HOST"),
        auth_plugin=os.environ.get("AUTH_PLUGIN")
        ) 

        cur = conn.cursor(prepared=True)
        mysql_query: str = """SELECT date_of_price FROM nordpool.HourlyPricesLT where date_of_price=%s;"""
        cur.execute(mysql_query, (params,))
        cur.fetchone()
        row_count = cur.rowcount
        if row_count > 0:       
            return True
        return False  
        
    except mysql.connector.Error as error:
        print("Failed to execute MySQL query {}".format(error)) # VIETA LOGINIMUI

    finally:
        if conn.is_connected():
            cur.close()
            conn.close()
            print("MySQL conn is closed.") #VIETA LOGINIMUI




In [5]:
#### ORIGINAL

## NP METHODS TO RETRIEVE DATA AND PUSH IT TO DB

# pip install mysql-connector-python
import os
import mysql.connector
from typing import List, Dict, Optional
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service # del netinkamos driverio versijos galima autoamtiskai sutvarkjyt
#from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, date
from bs4 import BeautifulSoup


#poetry add python-dotenv - cia iadziant su jupiter
#from dotenv import load_dotenv


def get_current_prices(soup = _get_page_content(query)) -> None:
    """Populate database with current data in main page."""

    all_dates = soup.find("tr", class_="column-headers").find_all("th")
    prices = soup.find("table", id="datatable").tbody.find_all("tr")    
    filtered_dates = _check_date_in_database(all_dates)
    counter: int = len(filtered_dates) # eiluciu su datomis skaicius
    
    if counter != 0:
        # Ripping dates off from the site     
        for date in filtered_dates:                     
            price_tags: List = [float]
                            
            for priceline in prices[:24]:            
                price_tag = float(priceline.find_all("td")[counter].text.replace(',', '.'))                      
                price_tags.append(price_tag)
                    
            mysql_query: str = '''INSERT INTO nordpool.HourlyPricesLT (date_of_price, time_00_01, time_01_02, time_02_03, time_03_04, time_04_05, time_05_06, time_06_07, time_07_08, time_08_09, time_09_10, time_10_11, time_11_12, time_12_13, time_13_14, time_14_15, time_15_16, time_16_17, time_17_18, time_18_19, time_19_20, time_20_21, time_21_22, time_22_23, time_23_24)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''
            params: tuple = (date, price_tags[1], price_tags[2], price_tags[3], price_tags[4], price_tags[5], price_tags[6], price_tags[7], price_tags[8], price_tags[9], price_tags[10], price_tags[11], price_tags[12], price_tags[13], price_tags[14], price_tags[15], price_tags[16], price_tags[17], price_tags[18], price_tags[19], price_tags[20], price_tags[21], price_tags[22], price_tags[23], price_tags[24])
            _push_to_database(mysql_query, params)
            counter -= 1
    else:
        print("No new records found.")


""" Old method
def get_current_prices___old(soup = _get_page_content(query)) -> None:
    '''Populate database with current data in main page.'''

    dates = soup.find("tr", class_="column-headers").find_all("th")
    prices = soup.find("table", id="datatable").tbody.find_all("tr")
    counter: int = len(dates) # stulpeliu su datomis skaicius
    


    # Ripping dates off from the site     
    for day_date in reversed(dates[1:counter]): 
        # Pakonvertuoja is 08-12-2022 i 2022-12-08 + pavercia i date tipa
        date_object = datetime.strptime(day_date.text, '%d-%m-%Y').date()            
        price_tags: List = [float]
        exists: bool = _check_date_in_database(date_object)

        if not exists:          
            for priceline in prices[:24]:            
                price_tag = float(priceline.find_all("td")[counter - 1].text.replace(',', '.'))                      
                price_tags.append(price_tag)
                    
            mysql_query: str = '''INSERT INTO nordpool.HourlyPricesLT (date_of_price, time_00_01, time_01_02, time_02_03, time_03_04, time_04_05, time_05_06, time_06_07, time_07_08, time_08_09, time_09_10, time_10_11, time_11_12, time_12_13, time_13_14, time_14_15, time_15_16, time_16_17, time_17_18, time_18_19, time_19_20, time_20_21, time_21_22, time_22_23, time_23_24)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''
            params: tuple = (date_object, price_tags[1], price_tags[2], price_tags[3], price_tags[4], price_tags[5], price_tags[6], price_tags[7], price_tags[8], price_tags[9], price_tags[10], price_tags[11], price_tags[12], price_tags[13], price_tags[14], price_tags[15], price_tags[16], price_tags[17], price_tags[18], price_tags[19], price_tags[20], price_tags[21], price_tags[22], price_tags[23], price_tags[24])
            _push_to_database(mysql_query, params)
            #counter -= 1
        else:
            #counter -= 1
            print(f"Date {date_object} already in.") #loginimui
        counter -= 1
"""   



test2 = get_current_prices()
test2






C:\Users\laimis\AppData\Local\Temp\ipykernel_19696\1618975745.py:28: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)   #options=options,


MySQL conn for checks is closed.
Records pushed succesfully
MySQL conn is closed.


In [24]:
### TESTING MySQL

def _push_to_database2(sql_queries: List, sql_params: List) -> None:
    """Methods retrieves year id from database. If nno such year in database returns NONE."""
    try:
        conn = mysql.connector.connect(
        user=os.environ.get("DB_USER"),
        password=os.environ.get("DB_PASSWORD"),
        database=os.environ.get("DATABASE"),
        port=int(os.environ.get("DB_PORT")),
        host=os.environ.get("DB_HOST"),
        auth_plugin=os.environ.get("AUTH_PLUGIN")
        ) 


        print("entered push to db method try section")

        cur = conn.cursor()

        #prepared=True

        for query in sql_queries:
            print(query)
            for param in sql_params:
                print(param)
                cur.execute(query, param)
                conn.commit()
                print("Records pushed succesfully") # VIETA LOGINIMUI
        

    except mysql.connector.Error as error:
        print("Failed to execute MySQL query {}".format(error)) # VIETA LOGINIMUI

    finally:
        if conn.is_connected():
            cur.close()
            conn.close()
            print("MySQL conn for push is closed.") #VIETA LOGINIMUI


def _check_date_in_database2(all_dates) -> List:
    """Method checks if date exists in database. Records wich does not exists returns as list."""
    filtered_dates: List = []

    try:
        conn = mysql.connector.connect(
        user=os.environ.get("DB_USER"),
        password=os.environ.get("DB_PASSWORD"),
        database=os.environ.get("DATABASE"),
        port=int(os.environ.get("DB_PORT")),
        host=os.environ.get("DB_HOST"),
        auth_plugin=os.environ.get("AUTH_PLUGIN")
        ) 

        cur = conn.cursor()        
        for date in all_dates[1:]:
            # Pakonvertuoja is 08-12-2022 i 2022-12-08 + pavercia i date tipa
            formated_date = datetime.strptime(date.text, '%d-%m-%Y').date()
            mysql_query: str = """SELECT date_of_price FROM nordpool.HourlyPricesLT_test where date_of_price=%s;"""
            cur.execute(mysql_query, (formated_date,))
            cur.fetchone()
            row_count = cur.rowcount
            if row_count < 0:   
                filtered_dates.append(formated_date)
        filtered_dates.reverse()
        return filtered_dates
        
    except mysql.connector.Error as error:
        print("Failed to execute MySQL query {}".format(error)) # VIETA LOGINIMUI

    finally:
        if conn.is_connected():
            cur.close()
            conn.close()
            print("MySQL conn for checks is closed.") #VIETA LOGINIMUI









In [69]:
kveris: str = "https://www.nordpoolgroup.com/en/Market-data1/Dayahead/Area-Prices/LT/Hourly/?view=table"
soup = _get_page_content(kveris)
datules = soup.find("tr", class_="column-headers").find_all("th")

datos = _check_date_in_database2(datules)

print(type(datos))
print(datos)

C:\Users\laimis\AppData\Local\Temp\ipykernel_15236\1618975745.py:28: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)   #options=options,


MySQL conn for checks is closed.
<class 'list'>
[]


In [26]:
### TESTING. . .


## NP METHODS TO RETRIEVE DATA AND PUSH IT TO DB

# pip install mysql-connector-python
import os
import mysql.connector
from typing import List, Dict, Optional
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service # del netinkamos driverio versijos galima autoamtiskai sutvarkjyt
#from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, date
from bs4 import BeautifulSoup


#poetry add python-dotenv - cia iadziant su jupiter
#from dotenv import load_dotenv


def get_current_prices2(soup = _get_page_content(query)) -> None:
    """Populate database with current data in main page."""

    all_dates = soup.find("tr", class_="column-headers").find_all("th")
    filtered_dates = _check_date_in_database2(all_dates)
    prices = soup.find("table", id="datatable").tbody.find_all("tr")    
    counter: int = len(filtered_dates) # eiluciu su datomis skaicius
    sql_queries: List[str] = []
    sql_params: List[tuple] = []

    if counter != 0:
        # Ripping dates off from the site     
        for day in filtered_dates:                     
            price_tags: List = [float]
                            
            for priceline in prices[:24]:            
                price_tag = float(priceline.find_all("td")[counter].text.replace(',', '.'))                      
                price_tags.append(price_tag)
                    
            #mysql_query: str = f'''INSERT INTO nordpool.HourlyPricesLT_test (date_of_price, time_00_01, time_01_02, time_02_03, time_03_04, time_04_05, time_05_06, time_06_07, time_07_08, time_08_09, time_09_10, time_10_11, time_11_12, time_12_13, time_13_14, time_14_15, time_15_16, time_16_17, time_17_18, time_18_19, time_19_20, time_20_21, time_21_22, time_22_23, time_23_24) VALUES ({str(day)}, {price_tags[1]}, {price_tags[2]}, {price_tags[3]}, {price_tags[4]}, {price_tags[5]}, {price_tags[6]}, {price_tags[7]}, {price_tags[8]}, {price_tags[9]}, {price_tags[10]}, {price_tags[11]}, {price_tags[12]}, {price_tags[13]}, {price_tags[14]}, {price_tags[15]}, {price_tags[16]}, {price_tags[17]}, {price_tags[18]}, {price_tags[19]}, {price_tags[20]}, {price_tags[21]}, {price_tags[22]}, {price_tags[23]}, {price_tags[24]});'''
            mysql_query: str = '''INSERT INTO nordpool.HourlyPricesLT_test (date_of_price, time_00_01, time_01_02, time_02_03, time_03_04, time_04_05, time_05_06, time_06_07, time_07_08, time_08_09, time_09_10, time_10_11, time_11_12, time_12_13, time_13_14, time_14_15, time_15_16, time_16_17, time_17_18, time_18_19, time_19_20, time_20_21, time_21_22, time_22_23, time_23_24) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'''
            params: tuple = (date, price_tags[1], price_tags[2], price_tags[3], price_tags[4], price_tags[5], price_tags[6], price_tags[7], price_tags[8], price_tags[9], price_tags[10], price_tags[11], price_tags[12], price_tags[13], price_tags[14], price_tags[15], price_tags[16], price_tags[17], price_tags[18], price_tags[19], price_tags[20], price_tags[21], price_tags[22], price_tags[23], price_tags[24])
            sql_queries.append(mysql_query)
            sql_params.append(params)
            counter -= 1

            
    else:
        print("No new records found.")
    #print(sql_query_list)
    _push_to_database2(sql_queries, sql_params)



test2 = get_current_prices2()
test2


C:\Users\laimis\AppData\Local\Temp\ipykernel_19696\1618975745.py:28: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)   #options=options,


MySQL conn for checks is closed.
entered push to db method try section
"INSERT INTO nordpool.HourlyPricesLT_test (date_of_price, time_00_01, time_01_02, time_02_03, time_03_04, time_04_05, time_05_06, time_06_07, time_07_08, time_08_09, time_09_10, time_10_11, time_11_12, time_12_13, time_13_14, time_14_15, time_15_16, time_16_17, time_17_18, time_18_19, time_19_20, time_20_21, time_21_22, time_22_23, time_23_24) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
(<class 'datetime.date'>, 79.4, 79.17, 80.0, 81.43, 81.94, 110.9, 62.25, 64.92, 86.01, 120.99, 137.64, 124.39, 108.59, 119.98, 151.19, 153.71, 171.03, 191.99, 199.13, 199.17, 174.71, 165.25, 157.35, 146.97)
Failed to execute MySQL query Failed processing format-parameters; Python 'type' cannot be converted to a MySQL type
MySQL conn for push is closed.
